In [11]:
from tensorflow.python.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
import numpy as np
from numpy import argmax
import os
import cv2 #이미지 읽기용
import matplotlib.pyplot as plt

In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
list = [[0]*4]*100 # 1 x 4 배열을 100개 만듬
count = 0

In [6]:
img = cv2.imread('face.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3,5)
for (x,y,w,h) in faces: #좌표 값과 rectangular의 width height를 받게 된다.
    # x,y값은 rectangular가 시작하는 지점의 좌표
    # x, y값은 좌표값 w,h는 얼굴 높이 너비
    # 원본 이미지에 얼굴의 위치를 표시하는 작업을 함.
    # for문을 돌리는 이유는 여러 개가 검출 될 수 있기 때문.

    list[count][0] = x
    list[count][1] = y
    list[count][2] = w
    list[count][3] = h
    print(list[count])
    #image 좌표값대로 잘라서 따로 저장하는 부분
    img_trim = img[y:y + h,x:x + w]  # trim한 결과를 img_trim에 담는다
    cv2.imwrite('target_{}.jpg'.format(count),img_trim) #각 카운터에 맞는 이름으로 저장
    
    count = count + 1

    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

[57, 96, 199, 199]
[260, 64, 190, 190]


In [7]:
img.shape

(330, 540, 3)

In [8]:
cv2.imshow('image',img)
cv2.waitKey()

-1

In [12]:
#분석 준비
image_size=32
labels=['neutral', 'fear', 'happy', 'sad', 'angry', 'surprise']
analy_image=[]
img_count = 0

#데이터 전처리
for index in faces:
    image = cv2.imread("target_{}.jpg".format(img_count)) #이미지 읽기
    image = cv2.resize(image, (image_size,image_size)) #이미지 사이즈 편집
    image = img_to_array(image) #이미지 배열화
    image = np.array(image, dtype="float64") / 255.0
    print(image.shape)
    analy_image.append(image)
    img_count += 1


(32, 32, 3)
(32, 32, 3)


In [ ]:
model = load_model('.h5')

In [ ]:
image_predict = model.predict(analy_image) #사진 분석

In [ ]:
#결과 출력
label = labels[image_predict[0].argmax()]
confidence = image_predict[0][image_predict[0].argmax()]
print('{} {:.2f}%'.format(label, confidence * 100)) 